# save data to json

In [25]:
import json
import numpy as np
import pandas as pd
import pickle
import scipy
from scipy import constants
import os, sys
from cmcrameri import cm

##plotting
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import corner
plt.style.use("Solarize_Light2")
plt.rcParams.update({"axes.edgecolor": "black"})
plt.rcParams.update({"text.color": "black"})
plt.rcParams.update({"axes.labelcolor": "black"})
plt.rcParams.update({"xtick.color": "black"})
plt.rcParams.update({"ytick.color": "black"})
plt.rcParams.update({"font.family": "monospace"})

In [2]:
## template for exemplars
star_name = 'powerpuff'

star_dict={}

star_dict['calc_effective_T']=[5904.7, 100]
star_dict['luminosity']=[1.159, 0.04]
star_dict['star_feh']=[-0.003, 0.1]

with open(f'powerpuff/{star_name}.json', 'w') as fp:
    json.dump(star_dict, fp)

In [26]:
def pbjam_to_json(star_name,
                 calc_effective_T = [5904.7, 100],
                 luminosity = [1.159, 0.04],
                 star_feh = [-0.003, 0.1],
                 ells = True):
    
    with open(f"powerpuff/{star_name}/{star_name}.pickle", 'rb') as fp:
        pbjam_out = pickle.load(fp)
    
    with open(f"powerpuff/{star_name}/{star_name}_ID.pickle", 'rb') as fp:
        pbjam_ID = pickle.load(fp)

    if ells:
        
        pbjam_l = pbjam_ID['ell']
        pbjam_n = pbjam_ID['enn']

        l0_n = pbjam_n[np.where(pbjam_l==0)]

        pbjam_n[np.where(pbjam_l==1)] = l0_n

        
        pbjam_freqs = pbjam_out['summary']['freq'][0]
        pbjam_freqs_err = pbjam_out['summary']['freq'][1]

    if not ells:
        l_eq0_idx = np.where(pbjam_ID['ell']==0)
        
        pbjam_l = pbjam_ID['ell'][l_eq0_idx]
        pbjam_n = pbjam_ID['enn'][l_eq0_idx]
        
        pbjam_freqs = pbjam_out['summary']['freq'][0][l_eq0_idx]
        pbjam_freqs_err = pbjam_out['summary']['freq'][1][l_eq0_idx]
        
    pbjam_numax = pbjam_ID['summary']['numax'][0]
    pbjam_numax_err = pbjam_ID['summary']['numax'][1]
    
    pbjam_dnu = pbjam_ID['summary']['dnu'][0]
    pbjam_dnu_err = pbjam_ID['summary']['dnu'][1]
    
    ## generate dict
    star_dict = {}
    
    star_dict['calc_effective_T']=calc_effective_T
    star_dict['luminosity']=luminosity
    star_dict['star_feh']=star_feh
    
    for i in range(len(pbjam_l)):
        ell = int(pbjam_l[i])
        enn = int(pbjam_n[i])
        
        header = f'nu_{ell}_{enn}'
    
        freq = pbjam_freqs[i]
        freq_err = pbjam_freqs_err[i]
        
        star_dict[header] = [freq, freq_err]
    
    star_dict['dnu'] = [pbjam_dnu, pbjam_dnu_err]
    star_dict['nu_max'] = [pbjam_numax, pbjam_numax_err]

    if ells:
        with open(f'powerpuff/{star_name}/{star_name}_l012.json', 'w') as fp:
            json.dump(star_dict, fp)

    if not ells:
        with open(f'powerpuff/{star_name}/{star_name}_l0.json', 'w') as fp:
            json.dump(star_dict, fp)

In [29]:
star_name = 'bubbles'

pbjam_to_json(star_name, ells=True)

pbjam_to_json(star_name, ells=False)